In [1]:
import pandas as pd ## For data manipulation
import numpy as np ## For data manipulation
from geopy.geocoders import Nominatim ## For Geo coding
import pandasql ## For data manipulation
import requests ## To retrive website data
import time

In [2]:
##Get longtitude and latitude for neighborhood near Reno based on Washoe county data https://www.washoecounty.us/assessor/online_data/DataDownloads.php

df = pd.read_csv('Quick_Info.txt', delimiter='\t')

nh = pandasql.sqldf("Select Neighborhood,SUM(AssessedTotal) as AssessedTotal,AVG(lat) as latitudes, AVG(lng) as longitudes, city From df  Where lat is not null and lng is not null and Neighborhood is not null and City in('RENO','SPARKS') Group by Neighborhood;", globals())

nh.head()
nh.shape

(1128, 5)

In [3]:
##Due to limitation of API we have to slpit data into 3 parts, run each of them through Foursquare API then UNION them back
nh1 = nh[0:400]
nh2 = nh[401:800]
nh3 = nh[801:1200]

print('nh1: ' + str(nh1.shape))
print('nh2: ' + str(nh2.shape))
print('nh3: ' + str(nh3.shape))

nh1: (400, 5)
nh2: (399, 5)
nh3: (327, 5)


In [4]:
##For later use, we write the output to local drive
nh1.to_csv(r'nh1.csv',index=None, header=True)
nh2.to_csv(r'nh2.csv',index=None, header=True)
nh3.to_csv(r'nh3.csv',index=None, header=True)

In [5]:
#Accessing Foursquare credential
CLIENT_ID = 'OQAAJJ3CT01KF0N4WPAMZXB210HKM4JKJ1FKS1TWEJ51JUBX' # your Foursquare ID
CLIENT_SECRET = 'P3W01TQDUMWOPPWMCS3H5WQCE1NE1UR2E0YABQHKXSNBTHFS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQAAJJ3CT01KF0N4WPAMZXB210HKM4JKJ1FKS1TWEJ51JUBX
CLIENT_SECRET:P3W01TQDUMWOPPWMCS3H5WQCE1NE1UR2E0YABQHKXSNBTHFS


In [6]:
#Create function to explore neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [8]:
##If there is error "KeyError: 'groups'", it means we reached limit for the day
to_venues1 = getNearbyVenues(names=nh1['Neighborhood'],
                            latitudes=nh1['latitudes'],
                            longitudes=nh1['longitudes'])
to_venues1.to_csv(r'venues1.csv',index=None, header=True)

ACAC
ACBD
ACCD
ACDC
ACED
ACFD
ACID
ACJC
ACKC
ACLE
ACMD
ACNE
ACOD
ACPE
ACQE
ACRE
ACSE
ACVV
ACYY
ADAA
ADAQ
ADAS
ADBS
ADCN
ADDK
ADFC
ADFK
ADHR
ADIA
ADJA
ADKB
ADMA
ADNA
ADPA
ADQA
ADRA
ADSA
ADTA
ADTF
ADVP
ADVV
ADYY
AEBK
AEEA
AEFP
AEJC
AEVV
AEYY
AFBK
AFCA
AFDA
AFEC
AFFQ
AFGB
AFHA
AFIA
AFJA
AFKC
AFLC
AFMC
AFNC
AFOD
AFPC
AFRE
AFSM
AFTA
AFUC
AFVE
AFVV
AFWC
AFXC
AFYY
AGAA
AGAC
AGCQ
AGFK
AGGD
AGHE
AGHK
AGIN
AGJQ
AGKA
AGMA
AGOU
AGPA
AGQA
AGRA
AGTS
AGUM
AGVV
AGWC
AGYY
AHBQ
AHCU
AHEK
AHFC
AHGA
AHIP
AHVV
AHYY
AIAP
AIDQ
AIEA
AIFA
AIGM
AIHC
AIIC
AIJD
AIVV
AIYY
AJAA
AJAQ
AJBA
AJBK
AJBQ
AJDA
AJGC
AJVV
AJYY
AKAQ
AKCM
AKDA
AKFN
AKVV
AKYY
ALAQ
ALBK
ALCM
ALFC
ALVV
ALYY
AMAQ
AMEZ
AMVV
AMYY
ANAE
ANAF
ANBF
ANCB
ANDP
ANFD
ANFE
ANGE
ANGW
ANHE
ANIC
ANJA
ANKA
ANLC
ANMC
ANND
ANOE
ANPD
ANQD
ANQQ
ANRA
ANTA
ANUC
ANVA
ANVV
ANYB
ANYY
ANZD
AOAS
AOBQ
AOCA
AODA
AOEA
AOFA
AOHA
AOIO
AOJC
AOKK
AOVV
AOYY
BAAA
BABA
BACA
BACK
BADQ
BAEB
BAFC
BAGC
BAGZ
BAHD
BAIC
BAJC
BALD
BAMD
BAVV
BAYY
BBAA
BBBA
BBBR
BBCQ
BBEK
BBFC
BBGB
BBHA
BBIC


In [8]:
##If there is error "KeyError: 'groups'", it means we reached limit for the day
to_venues2 = getNearbyVenues(names=nh2['Neighborhood'],
                            latitudes=nh2['latitudes'],
                            longitudes=nh2['longitudes'])
to_venues2.to_csv(r'venues2.csv',index=None, header=True)

DKDC
DKEB
DKFC
DKGC
DKHF
DKIE
DKJE
DKKC
DKLC
DKMC
DKVV
DKYY
DLAA
DLAB
DLAD
DLBC
DLCB
DLCC
DLDA
DLDC
DLEC
DLFC
DLFD
DLGC
DLGD
DLGO
DLGP
DLGQ
DLGW
DLHC
DLIC
DLJC
DLKC
DLKZ
DLLC
DLMC
DLVV
DLYY
DMAD
DMAE
DMBB
DMBD
DMCC
DMCD
DMDA
DMDB
DMDC
DMEA
DMEB
DMED
DMFC
DMFD
DMGC
DMGD
DMGW
DMID
DMJQ
DMKH
DMMA
DMOF
DMVV
DMYY
DNAC
DNAD
DNBC
DNBQ
DNVV
DNYY
DOAC
DOYY
DPAB
DPAC
DPBQ
DPDB
DPVV
DPYY
DQAC
DQAQ
DQAZ
DQEZ
DQVV
DQYY
EAAF
EABA
EABD
EABE
EABF
EABG
EABQ
EACD
EADE
EADZ
EAEZ
EAFF
EAFQ
EAFR
EAGC
EAGD
EAGF
EAGG
EAHF
EAJE
EAVV
EAYY
EBAD
EBAQ
EBBD
EBDD
EBED
EBFD
EBGE
EBHD
EBLE
EBVV
EBZZ
ECAT
ECBE
ECCE
ECCF
ECDE
ECDF
ECEC
ECED
ECFD
ECFN
ECFQ
ECFS
ECFU
ECFZ
ECGF
ECHE
ECID
ECJG
ECKF
ECLF
ECMG
ECNG
ECOF
ECPC
ECQB
ECRB
ECVV
ECWW
ECYY
EDAB
EDAC
EDAD
EDBB
EDBC
EDBF
EDBG
EDCQ
EDDC
EDEB
EDEC
EDGC
EDGE
EDHC
EDHD
EDHE
EDIA
EDIC
EDJB
EDKC
EDKF
EDLC
EDMC
EDNC
EDOC
EDOZ
EDPC
EDPZ
EDVV
EDWW
EDYY
EEAA
EEBC
EECA
EEFB
EEGC
EEIC
EEVV
EEYY
EFCB
EFDC
EFFA
EFIA
EFLQ
EFMA
EFNC
EFOO
EFPP
EFQT
EFRU
EFVV
EFYY
EFZZ
EGAJ
FAAB
FAAC


In [8]:
##If there is error "KeyError: 'groups'", it means we reached limit for the day
to_venues3 = getNearbyVenues(names=nh3['Neighborhood'],
                            latitudes=nh3['latitudes'],
                            longitudes=nh3['longitudes'])
to_venues3.to_csv(r'venues3.csv',index=None, header=True)

IAFQ
IAVV
IAYY
IBAQ
IBCI
IBDH
IBEJ
IBGF
IBHF
IBHH
IBVV
IBWW
IBYY
ICCZ
IDBF
IDVV
IDWW
IDYY
IEAC
IEBC
IECQ
IEDD
IEEE
IEGF
IEHN
IEJF
IELG
IEMF
IENF
IEVV
IEWW
IEYY
IGAF
IGAG
IGBG
IGBH
IGCG
IGDH
IGGW
IGJJ
IGVV
IGWW
IGYY
IHAZ
IHVV
JAGG
JAHJ
JBCQ
JBDZ
JBEF
JBFI
JBVV
JBYY
JCAH
JCAZ
JCBF
JCCG
JCEF
JCFF
JCGF
JCGW
JCHD
JCHF
JCIF
JCJD
JCKE
JCLF
JCMF
JCNQ
JCVV
JCWW
JCYY
JCZZ
KAAZ
KAWW
KBAF
KBAN
KBBQ
KBBZ
KBWW
KBYY
LAAC
LAAD
LAAF
LAAJ
LAAQ
LAAU
LAVV
LAWW
LAYY
LBAC
LBAJ
LBAZ
LBWW
MAAJ
MABJ
MABZ
MACH
MAEJ
MAFJ
MAWW
MAZZ
MMMM
NAAQ
NAAU
NABA
NACA
NADA
NAEC
NAFK
NAHF
NAIA
NAMA
NAVV
NAYY
NBBP
NBCQ
NBDS
NBEA
NBYY
NCAR
NCCU
NCDT
NCEU
NCGY
NCHT
NCVV
NCWW
NCYY
NDBA
NDCA
NDEQ
NDFB
NDGB
NDHN
NDIC
NDJB
NDKF
NDLB
NDMC
NDVV
NDWW
NDYY
NEAJ
NEBD
NECA
NEDF
NEFE
NEGW
NEJE
NEVV
NEYY
NFBJ
NGAN
NGAQ
NGAU
NGBU
NGFT
NGGC
NGGK
NGHQ
NGVV
NGYY
OAAD
OABD
OACA
OADD
OAEG
OAFD
OAGB
OAGW
OAHC
OAIC
OAJA
OAKC
OAPB
OAQA
OARC
OASD
OATC
OAUB
OAVD
OAVV
OAWZ
OAYY
OBAA
OBCT
OBDP
OBEO
OBFD
OBGQ
OBHU
OBVV
OBWW
OBYY
OCAA
OCBB
OCCD
OCDF
OCEC


In [8]:
##Reading the API output then UNION them together then save the final Data Frame for later use
to_venues1 = pd.read_csv('venues1.csv', delimiter = ',')
to_venues2 = pd.read_csv('venues2.csv', delimiter = ',')
to_venues3 = pd.read_csv('venues3.csv', delimiter = ',')

to_venues = pd.concat([to_venues1, to_venues2, to_venues3], ignore_index=True)
to_venues.to_csv(r'venues.csv',index=None, header=True)

In [9]:
##Reading the final output from previous step. If you see the steps are not following the order because I ran the API steps again to remove the error.
to_venues = pd.read_csv('venues.csv', delimiter = ',')
to_venues.shape
to_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ACAC,39.486542,-119.850716,Raley’s,39.483828,-119.852709,Grocery Store
1,ACAC,39.486542,-119.850716,Starbucks,39.484517,-119.853007,Coffee Shop
2,ACAC,39.486542,-119.850716,Anytime Fitness,39.483270,-119.854600,Gym / Fitness Center
3,ACAC,39.486542,-119.850716,Wells Fargo Bank,39.483694,-119.855218,Bank
4,ACAC,39.486542,-119.850716,Moxie's Spirits & Dining,39.483610,-119.852479,American Restaurant


In [10]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")
to_onehot.head()

,ATM,Accessories Store,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# add Neighborhood column back to dataframe
to_onehot['NH'] = to_venues['Neighborhood']

In [12]:
# move Neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,NH,ATM,Accessories Store,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACAC,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#group rows by NH and by taking the mean of the frequency of occurrence of each category
to_grouped = to_onehot.groupby('NH').mean().reset_index()
to_grouped

,NH,ATM,Accessories Store,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ACAC,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACBD,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACDC,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACED,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACFD,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,TAUD,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
867,TAVE,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,TAVV,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,TAYY,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Create function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['NH']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nh_venues_sorted = pd.DataFrame(columns=columns)
nh_venues_sorted['NH'] = to_grouped['NH']

for ind in np.arange(to_grouped.shape[0]):
    nh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

nh_venues_sorted.head()

,NH,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ACAC,Grocery Store,Bank,Gym / Fitness Center,American Restaurant,Coffee Shop,Flea Market,Field,Financial or Legal Service,Fireworks Store,Fishing Spot
1,ACBD,Ice Cream Shop,Grocery Store,American Restaurant,Fondue Restaurant,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop
2,ACDC,Ice Cream Shop,Fondue Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food
3,ACED,Italian Restaurant,Scenic Lookout,Yoga Studio,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market
4,ACFD,Scenic Lookout,Yoga Studio,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Fondue Restaurant


In [16]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

to_grouped_clustering = to_grouped.drop('NH', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 6, 6, 6, 6, 6, 5, 6, 6, 6], dtype=int32)

In [17]:
# add clustering labels
nh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [18]:
to_merged = nh

# merge toronto grouped with toronto df to add latitude/longitude for each neighborhood
to_merged = to_merged.join(nh_venues_sorted.set_index('NH'), on='Neighborhood')
to_merged.head()

,Neighborhood,AssessedTotal,latitudes,longitudes,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ACAC,10511213.0,39.486542,-119.850716,RENO,6.0,Grocery Store,Bank,Gym / Fitness Center,American Restaurant,Coffee Shop,Flea Market,Field,Financial or Legal Service,Fireworks Store,Fishing Spot
1,ACBD,10827371.0,39.482518,-119.847469,RENO,6.0,Ice Cream Shop,Grocery Store,American Restaurant,Fondue Restaurant,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop
2,ACCD,12084462.0,39.479323,-119.842410,RENO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACDC,10284040.0,39.484266,-119.842608,RENO,6.0,Ice Cream Shop,Fondue Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food
4,ACED,19717135.0,39.488588,-119.841727,RENO,6.0,Italian Restaurant,Scenic Lookout,Yoga Studio,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market


In [19]:
#Fix datatype
to_merged = to_merged.replace([np.inf, -np.inf], np.nan)
to_merged = to_merged.dropna(subset = ['Cluster Labels'])
to_merged = to_merged.astype({'Cluster Labels': 'int32'})
to_merged.dtypes

Neighborhood               object
AssessedTotal             float64
latitudes                 float64
longitudes                float64
City                       object
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [20]:
# Matplotlib and associated plotting modules
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

#Target location
latitude = 	39.530895
longitude = -119.814972

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['latitudes'], to_merged['longitudes'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [21]:
#Based on clustering we see that Neighboor Hood in cluster 5 and 6 has restaurant/coffee shop as their 1st common venue. We can exclude cluster 5 & 6 (light teal and green) and focus on the remaining neighboor cluster to open a restaurant.
to_merged[(to_merged['Cluster Labels'] != 6) & (to_merged['Cluster Labels'] != 5)]

,Neighborhood,AssessedTotal,latitudes,longitudes,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,ACQE,13976621.0,39.499778,-119.846521,RENO,2,Park,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Yoga Studio
47,AEYY,1337819.0,39.514349,-119.815098,RENO,2,Gym,Park,Flea Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Yoga Studio
66,AFUC,22959937.0,39.509304,-119.828192,RENO,1,Home Service,Yoga Studio,Fondue Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop
70,AFXC,10400830.0,39.513750,-119.818738,RENO,2,Gym,Park,Flea Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Yoga Studio
128,ALFC,53972795.0,39.514863,-119.795264,RENO,2,Park,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Yoga Studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,RBID,14801440.0,39.477129,-119.850790,RENO,7,Construction & Landscaping,Yoga Studio,Food,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant
1084,RBJC,21899120.0,39.494839,-119.863072,RENO,2,Park,Trail,Gym,Food Stand,Fishing Spot,Farmers Market,Fast Food Restaurant,Food Truck,Field,Financial or Legal Service
1087,RBME,42667822.0,39.491829,-119.871407,RENO,8,Art Gallery,Gym,Yoga Studio,Fondue Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market
1097,RBVV,95992.0,39.490977,-119.866389,RENO,8,Gym,Sporting Goods Shop,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fishing Spot,Fishing Store,Flea Market
